# Context

>[Context](https://context.ai/) 为LLM驱动的产品和功能提供用户分析。

使用`Context`，您可以在30分钟内开始了解您的用户并改善他们的体验。


在本指南中，我们将向您展示如何与Context集成。

## 安装和设置

In [ ]:
%pip install --upgrade --quiet  langchain langchain-openai langchain-community context-python

### 获取API凭证

获取Context API令牌的步骤：

1. 进入您的Context账户中的设置页面 (https://with.context.ai/settings)。
2. 生成一个新的API令牌。
3. 将此令牌存储在安全的地方。

### 设置Context

要使用`ContextCallbackHandler`，从Langchain导入处理程序并使用您的Context API令牌实例化它。

在使用处理程序之前，请确保您已安装`context-python`包。

In [1]:
from langchain_community.callbacks.context_callback import ContextCallbackHandler

In [3]:
import os

token = os.environ["CONTEXT_API_TOKEN"]

context_callback = ContextCallbackHandler(token)

## 使用方法
### 聊天模型中的Context回调

Context回调处理程序可用于直接记录用户和AI助手之间的对话记录。

In [ ]:
import os

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

token = os.environ["CONTEXT_API_TOKEN"]

chat = ChatOpenAI(
    headers={"user_id": "123"}, temperature=0, callbacks=[ContextCallbackHandler(token)]
)

messages = [
    SystemMessage(
        content="你是一个将英语翻译成法语的助手。"
    ),
    HumanMessage(content="I love programming."),
]

print(chat(messages))

### 链中的Context回调

Context回调处理程序也可用于记录链的输入和输出。请注意，不会记录链的中间步骤——只记录初始输入和最终输出。

__注意：__ 确保您将相同的context对象传递给聊天模型和链。

错误做法：
> ```python
> chat = ChatOpenAI(temperature=0.9, callbacks=[ContextCallbackHandler(token)])
> chain = LLMChain(llm=chat, prompt=chat_prompt_template, callbacks=[ContextCallbackHandler(token)])
> ```

正确做法：
>```python
>handler = ContextCallbackHandler(token)
>chat = ChatOpenAI(temperature=0.9, callbacks=[callback])
>chain = LLMChain(llm=chat, prompt=chat_prompt_template, callbacks=[callback])
>```


In [ ]:
import os

from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

token = os.environ["CONTEXT_API_TOKEN"]

human_message_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="为一家生产{product}的公司取一个好名字？",
        input_variables=["product"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
callback = ContextCallbackHandler(token)
chat = ChatOpenAI(temperature=0.9, callbacks=[callback])
chain = LLMChain(llm=chat, prompt=chat_prompt_template, callbacks=[callback])
print(chain.run("五彩袜子"))